In [19]:
import sys
sys.path.append("/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/ThinkDSP/code")
sys.path.append("/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/")
sys.path.append("Users/luigibungaro/dsr/Audio_Probreat/")

In [20]:
from pyAudioAnalysis import audioFeatureExtraction as aFE, audioBasicIO,\
audioAnalysis,audioSegmentation,audioVisualization,audioTrainTest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import numpy as np
import keras
from keras import models,layers
import os
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, GRU
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from IPython.display import HTML

FIR (finite impulse response) filter

#### • Band-pass filter 
attenuates both high an low
frequencies and passes only an intermediate range of
frequencies
#### • High pass filter 
passes high-frequency signals, but
attenuates signals with frequencies lower than the cutoff
frequency
#### • Low pass filter 
passes low frequency signals and
attenuates frequencies higher than the cutoff frequency
#### • Notch filter: band-stop filter with a narrow stop band
(passes most frequencies, but attenuates thos within
the stop band) – usually used for the 50Hz of power
lines

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [21]:
def amplitude_data_all():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur' or '/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_murmur/'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_murmur/
                                                   '+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal' or '/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_normal/'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'or '/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/pyAudioAnalysis/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
          
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [22]:
def amplitude_data_sample_mean(step_of_sampling):
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = pd.DataFrame(data)
    data.rolling(window = step_of_sampling, axis = 1).mean().iloc[:,range(step_of_sampling-1,len(data[0]),step_of_sampling)]
    #data.groupby(lambda x: x // 10, axis = 1).mean()
    data = np.array(data)
    data = data[:,0::step_of_sampling]
    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [23]:
def amplitude_data_sample_asis(step_of_sampling):
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    #sampling_indexes = np.arange(0,threshold,step_of_sampling)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        
    data = np.array(data)
    data = data[:,0::step_of_sampling]

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [24]:
def amplitude_data_difference():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                #X_temp = X[1:]
                #data.append(X_temp-X[:-1])
                data.append(data)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        #data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [25]:
X_train,X_test,y_train,y_test = amplitude_data_all()

FileNotFoundError: [Errno 2] No such file or directory: 'peterbertley_Audio/Atraining_murmur'

In [26]:
X_train,X_test,y_train,y_test = amplitude_data_sample_asis(50)

FileNotFoundError: [Errno 2] No such file or directory: 'peterbertley_Audio/Atraining_murmur'

In [27]:
X_train,X_test,y_train,y_test = amplitude_data_sample_mean(50)

FileNotFoundError: [Errno 2] No such file or directory: 'peterbertley_Audio/Atraining_murmur'

In [28]:
X_train,X_test,y_train,y_test = amplitude_data_difference()

FileNotFoundError: [Errno 2] No such file or directory: 'peterbertley_Audio/Atraining_murmur'

## ICA

In [17]:
print(__doc__)

# Authors: Alexandre Gramfort, Gael Varoquaux
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, FastICA

# #############################################################################
# Generate sample data
#rng = np.random.RandomState(42)
#S = rng.standard_t(1.5, size=(20000, 2))
#S[:, 0] *= 2.

# Mix data
#A = np.array([[1, 1], [0, 2]])  # Mixing matrix

#X = np.dot(S, A.T)  # Generate observations

pca = PCA()
y_pca_ = pca.fit(X_train).transform(X_train)

ica = FastICA(random_state=rng)
y_ica_ = ica.fit(y_train).transform(y_train)  # Estimate the sources

y_ica_ /= S_ica_.std(axis=0)


# #############################################################################
# Plot results

def plot_samples(y_train, axis_list=None):
    plt.scatter(S[:, 0], S[:, 1], s=2, marker='o', zorder=10,
                color='steelblue', alpha=0.5)
    if axis_list is not None:
        colors = ['orange', 'red']
        for color, axis in zip(colors, axis_list):
            axis /= axis.std()
            x_axis, y_axis = axis
            # Trick to get legend to work
            plt.plot(0.1 * x_axis, 0.1 * y_axis, linewidth=2, color=color)
            plt.quiver(0, 0, x_axis, y_axis, zorder=11, width=0.01, scale=6,
                       color=color)

    plt.hlines(0, -3, 3)
    plt.vlines(0, -3, 3)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.xlabel('x')
    plt.ylabel('y')

plt.figure()
plt.subplot(2, 2, 1)
plot_samples(y_train / y_train.std())
plt.title('True Independent Sources')

axis_list = [pca.components_.T, ica.mixing_]
plt.subplot(2, 2, 2)
plot_samples(X / np.std(X_train), axis_list=axis_list)
legend = plt.legend(['PCA', 'ICA'], loc='upper right')
legend.set_zorder(100)

plt.title('Observations')

plt.subplot(2, 2, 3)
plot_samples(y_pca_ / np.std(y_pca_, axis=0))
plt.title('PCA recovered signals')

plt.subplot(2, 2, 4)
plot_samples(y_ica_ / np.std(y_ica_))
plt.title('ICA recovered signals')

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.36)
plt.show()

Automatically created module for IPython interactive environment


NameError: name 'X_train' is not defined